给定一个RPA流程需求描述，先让LLM重新描述一个需求和执行步骤，然后生成伪代码，再根据API文档和伪代码，生成实际的代码。

With GPT-4

In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory


load_dotenv("../openai.env")
llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0.7, max_tokens=3000)

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', chunk_size=1)
docsearch = Chroma(persist_directory='apidocs', embedding_function=embeddings)

In [3]:
template_task = '''你是个非常优秀的RPA程序员，你的任务是根据RPA流程的需求描述和步骤描述，生成一个伪代码表示的RPA流程。
请一步步思考，确保正确理解实现该流程的所有必要条件，然后更新步骤描述，确保新的步骤描述包含了所有的上下文信息，以方便后面的代码。

流程描述: {description}

任务步骤：{steps}

重新描述实现该RPA流程的具体步骤，包括每个步骤的输入输出和实现功能：'''

prompt_task = PromptTemplate(input_variables=['description', 'steps'], template=template_task)
task_chain = LLMChain(llm=llm, prompt=prompt_task, output_key='revised')

In [4]:
template_draft = '''你是个非常优秀的RPA程序员，请根据RPA流程描述和实现步骤描述，先生成一个用伪代码表示的RPA流程草稿。

流程描述: {description}

详细步骤：{revised}

生成的伪代码RPA流程是：'''

prompt_draft = PromptTemplate(input_variables=['description', 'revised'], template=template_draft)
draft_chain = LLMChain(llm=llm, prompt=prompt_draft, output_key='draft')

In [5]:
template_code = '''你是个非常优秀的RPA程序员，请根据RPA流程描述和流程草稿，实现一个由基本操作组成的伪代码RPA流程。
基本操作是指计算机能够执行的最简单任务，比如打开文件、保存文件、打开浏览器、点击按钮、修改文本等。

流程描述: {description}
流程草稿: {draft}

生成的RPA流程伪代码：'''

prompt_code = PromptTemplate(input_variables=['description', 'draft'], template=template_code)

code_chain = LLMChain(llm=llm, prompt=prompt_code, output_key='code')

In [6]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[task_chain, draft_chain, code_chain],
    input_variables=['description', 'steps'],
    # Here we return multiple variables
    output_variables=["revised", "draft", "code"],
    verbose=True)

In [7]:
task_description = '''通过RPA机器人，进入网银系统获取前一个工作日的资金流水数据，
按数据类型写入到模板表对应的银行列中，并下载前一工作日电子回单，生成文件清单并打印，
再发送资金日报及电子回单清单给对应的业务人员。
'''

task_steps = '''操作步骤及说明
1.	业务人员配置好银行账号数据表，和邮箱账号数据；
2.	读取账号配置表，根据账号信息自动登录网银系统；
3.	获取前一天历史余额数据，写入资金日报表模板对应银行账户余额栏；
4.	下载前一天电子回单、网银流水，按照固定规则命名（命名规则：公司_收/付_账号_日期，例：X公司_收_账号_20211222），并在日志记录表中写入记录；
5.	将日报excel及回、流水文件夹进行压缩；
6.	邮件发送运行异常提醒，将运行的数据压缩文件发送到指定邮箱；
7.	业务人员接收邮件文件，进行确认操作。
'''

In [8]:
result = overall_chain({'description': task_description, 'steps': task_steps})



> Entering new SequentialChain chain...

> Finished chain.


In [9]:
print(result['revised'])

1.	输入：业务人员配置的银行账号数据表，和邮箱账号数据。
   功能：准备账号信息，以用于后续的网银系统登录和邮件发送。
   输出：存储好的账号信息。

2.	输入：上一步存储的账号信息。
   功能：使用账号信息，自动登录网银系统。
   输出：登录成功的网银系统。

3.	输入：网银系统。
   功能：从网银系统获取前一天的历史余额数据。
   输出：前一天的历史余额数据。

4.	输入：前一天的历史余额数据，资金日报表模板。
   功能：将历史余额数据写入资金日报表模板对应的银行账户余额栏。
   输出：更新后的资金日报表。

5.	输入：网银系统。
   功能：从网银系统下载前一天的电子回单和网银流水，并按照规定规则命名，同时在日志记录表中写入记录。
   输出：下载的电子回单和网银流水，更新的日志记录表。

6.	输入：日报excel，电子回单和网银流水文件。
   功能：将这些文件进行压缩。
   输出：压缩后的文件。

7.	输入：压缩后的文件，存储的邮箱账号信息。
   功能：邮件发送运行异常提醒，将压缩文件发送到指定邮箱。
   输出：发送成功的邮件。

8.	输入：邮件。
   功能：业务人员接收邮件文件，进行确认操作。
   输出：业务人员的确认。


In [10]:
print(result['draft'])

```pseudocode
START
INITIALIZE accountInformation FROM businessData
INITIALIZE emailAccount FROM emailData

LOGIN bankingSystem WITH accountInformation
  IF loginSuccess THEN
     GET previousDayBalance FROM bankingSystem
     WRITE previousDayBalance INTO templateBankColumn IN reportTable
     DOWNLOAD previousDayReceipt AND bankingFlow FROM bankingSystem
        IF downloadSuccess THEN
           WRITE downloadLog INTO logTable
           COMPRESS reportTable, previousDayReceipt, bankingFlow INTO compressedFile
           SEND compressedFile TO emailAccount
        ELSE 
           PRINT "Download failed!"
        END IF
  ELSE 
     PRINT "Login failed!"
  END IF
END
```


In [11]:
print(result['code'])

```pseudocode
START

# Initialize account information and email account
SET accountInformation = GET businessData
SET emailAccount = GET emailData

# Login to the banking system
CALL Login(bankingSystem, accountInformation) 

IF loginSuccess THEN
     # Get the balance of the previous day from the banking system
     SET previousDayBalance = GET Balance(bankingSystem, previousDay)

     # Write the balance into the template bank column in the report table
     CALL Write(reportTable, templateBankColumn, previousDayBalance)

     # Download the receipt and banking flow from the previous day
     CALL Download(bankingSystem, previousDayReceipt, bankingFlow)

     IF downloadSuccess THEN
           # Write the download log into the log table
           CALL Write(logTable, downloadLog)

           # Compress the report table, receipt, and banking flow into a compressed file
           CALL Compress(reportTable, previousDayReceipt, bankingFlow, compressedFile)

           # Send the compre

In [23]:
# with open('patents/requirements-test4.txt', 'w') as f:
#     f.write('requirements: ' + result['requirements'])
#     f.write('\n\nfeature: ' + result['feature'])
#     f.write('\n\nsolution: ' + result['solution'])
#     f.write('\n\nselection: ' + result['selection'])

In [15]:
from langchain.chains import RetrievalQA


template_generate = """你是一个专业的RPA开发人员，请根据RPA库函数描述，选择合适的函数来生成一个步骤的代码。
如果实现这个任务步骤所需要的函数不存在于上下文中，则可以使用通用第三方函数库。如果存在，则必须使用已有的RPA库函数。
在生成代码时，只需要生成代码和注释，不需要别的解释。所有的注释都必须是中文。

相关函数: {context}

当前步骤：{question}
"""

prompt_generate = PromptTemplate(
    template=template_generate, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": prompt_generate}
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs)

In [16]:
steps = result['code'].split('\n\n')
steps[:3]

['```pseudocode\nSTART',
 '# Initialize account information and email account\nSET accountInformation = GET businessData\nSET emailAccount = GET emailData',
 '# Login to the banking system\nCALL Login(bankingSystem, accountInformation) ']

In [17]:
code_block = '# start'
for i in range(1, len(steps)):
    content = steps[i] + "\n\n please complete the code:\n" + code_block
    code_block += qa.run(steps[i])
    print(f'>> {steps[i]} \n {code_block} \n')

>> ```pseudocode
START 
 # start# 由于当前步骤为空，因此没有可以生成的代码。 

>> # Initialize account information and email account
SET accountInformation = GET businessData
SET emailAccount = GET emailData 
 # start# 由于当前步骤为空，因此没有可以生成的代码。# 使用RPA库函数receiveExchangeEmail接收邮件
emails = receiveExchangeEmail(
    serverAddress=emailAccount['serverAddress'], 
    adDomain=emailAccount['adDomain'], 
    username=emailAccount['username'], 
    password=emailAccount['password'], 
    emailAddress=emailAccount['emailAddress'], 
    folder='Inbox', 
    senderFilter=None, 
    subjectFilter=None, 
    contentFilter=None, 
    onlyUnread=True, 
    sortByDate=True, 
    attachmentSavePath='./attachments', 
    markAsRead=True, 
    numberOfEmails=10
)
# 注释：通过Exchange邮箱接收邮件，只接收未读邮件，最多接收10封，接收到的邮件按日期排序，邮件的附件存储到'./attachments'目录，接收后将邮件标记为已读。 

>> # Login to the banking system
CALL Login(bankingSystem, accountInformation)  
 # start# 由于当前步骤为空，因此没有可以生成的代码。# 使用RPA库函数receiveExchangeEmail接收邮件
emails = receiveExchangeEmail(
  